__Export Gold Data Set into SQL__ 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import pandas as pd

gold_Customers = pd.read_csv("gold.dim_customers.csv")



In [7]:
gold_Customers.head()

,customer_key,customer_id,customer_number,first_name,last_name,country,marital_status,gender,birthdate,create_date
0,1,11000,AW00011000,Jon,Yang,Australia,Married,Male,1971-10-06,2025-10-06
1,2,11001,AW00011001,Eugene,Huang,Australia,Single,Male,1976-05-10,2025-10-06
2,3,11002,AW00011002,Ruben,Torres,Australia,Married,Male,1971-02-09,2025-10-06
3,4,11003,AW00011003,Christy,Zhu,Australia,Single,Female,1973-08-14,2025-10-06
4,5,11004,AW00011004,Elizabeth,Johnson,Australia,Single,Female,1979-08-05,2025-10-06


In [9]:
gold_Customers.shape

(18484, 10)

In [10]:
gold_Customers.isna().sum()

customer_key         0
customer_id          0
customer_number      0
first_name           0
last_name            0
country            337
marital_status       0
gender              15
birthdate           17
create_date          0
dtype: int64

__Gold Products Analysis__

In [11]:
gold_products = pd.read_csv("gold.dim_products.csv")

In [12]:
gold_products.head()

,product_key,product_id,product_number,product_name,category_id,category,subcategory,maintenance,cost,product_line,start_date
0,1,210,FR-R92B-58,HL Road Frame - Black- 58,CO_RF,Components,Road Frames,Yes,0,Road,2003-07-01
1,2,211,FR-R92R-58,HL Road Frame - Red- 58,CO_RF,Components,Road Frames,Yes,0,Road,2003-07-01
2,3,348,BK-M82B-38,Mountain-100 Black- 38,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01
3,4,349,BK-M82B-42,Mountain-100 Black- 42,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01
4,5,350,BK-M82B-44,Mountain-100 Black- 44,BI_MB,Bikes,Mountain Bikes,Yes,1898,Mountain,2011-07-01


In [13]:
gold_products.shape

(295, 11)

In [14]:
gold_products.isna().sum()

product_key        0
product_id         0
product_number     0
product_name       0
category_id        0
category           7
subcategory        7
maintenance        7
cost               0
product_line      17
start_date         0
dtype: int64

__Gold sales Analysis__

In [15]:
Gold_Sales = pd.read_csv("gold.fact_sales.csv")

In [16]:
Gold_Sales.head()

,order_number,product_key,customer_key,order_date,shipping_date,due_date,sales_amount,quantity,price
0,SO54496,282,5400,2013-03-16,2013-03-23,2013-03-28,25,1,25
1,SO54496,289,5400,2013-03-16,2013-03-23,2013-03-28,5,1,5
2,SO54496,259,5400,2013-03-16,2013-03-23,2013-03-28,2,1,2
3,SO54497,174,9281,2013-03-16,2013-03-23,2013-03-28,22,1,22
4,SO54497,280,9281,2013-03-16,2013-03-23,2013-03-28,9,1,9


In [17]:
Gold_Sales.shape

(60398, 9)

In [18]:
Gold_Sales.isna().sum()

order_number      0
product_key       0
customer_key      0
order_date       19
shipping_date     0
due_date          0
sales_amount      0
quantity          0
price             0
dtype: int64

__Export Data CSV file to SQL__

__Gold Customer File__

In [26]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# 1️⃣ File & database details
csv_file = r"C:\path\to\gold.dim_customers.csv"  # <-- apna actual path yahan daalo
db_name = "SQL_Project"
db_user = "postgres"   # apna PostgreSQL username
db_password = "Password"
db_host = "localhost"
db_port = "5432"
table_name = "gold_customers"  # lowercase table name

# 2️⃣ Connect to default 'postgres' database to create new database if needed
conn = psycopg2.connect(
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,
    dbname="postgres"
)
conn.autocommit = True
cur = conn.cursor()

# 3️⃣ Check if database exists
cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{db_name}'")
exists = cur.fetchone()

# 4️⃣ Create database if it doesn't exist
if not exists:
    cur.execute(f'CREATE DATABASE "{db_name}"')
    print(f"Database '{db_name}' created!")
else:
    print(f"Database '{db_name}' already exists.")

cur.close()
conn.close()

# 5️⃣ Read CSV into pandas DataFrame
df = pd.read_csv("gold.dim_customers.csv")

# 6️⃣ Create SQLAlchemy engine for the new/existing database
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# 7️⃣ Export DataFrame to PostgreSQL (lowercase table)
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"CSV file '{csv_file}' has been successfully imported into PostgreSQL table '{table_name}'!")


Database 'SQL_Project' already exists.
CSV file 'C:\path\to\gold.dim_customers.csv' has been successfully imported into PostgreSQL table 'gold_customers'!


__Gold Product CSV File Export into SQL__

In [29]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# 1️⃣ File & database details
csv_file = "gold.dim_products.csv"  # <-- apna actual path yahan daalo
db_name = "SQL_Project"
db_user = "postgres"   # apna PostgreSQL username
db_password = "Password"
db_host = "localhost"
db_port = "5432"
table_name = "gold_products"  # lowercase table name, no quotes needed in SQL

# 2️⃣ Connect to default 'postgres' database to create new database if needed
conn = psycopg2.connect(
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,
    dbname="postgres"
)
conn.autocommit = True
cur = conn.cursor()

# 3️⃣ Check if database exists
cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{db_name}'")
exists = cur.fetchone()

# 4️⃣ Create database if it doesn't exist
if not exists:
    cur.execute(f'CREATE DATABASE "{db_name}"')
    print(f"Database '{db_name}' created!")
else:
    print(f"Database '{db_name}' already exists.")

cur.close()
conn.close()

# 5️⃣ Read CSV into pandas DataFrame
df = pd.read_csv(csv_file)

# 6️⃣ Convert table name to lowercase to avoid quotes in SQL
table_name = table_name.lower()

# 7️⃣ Create SQLAlchemy engine for the new/existing database
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# 8️⃣ Export DataFrame to PostgreSQL (lowercase table)
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"CSV file '{csv_file}' has been successfully imported into PostgreSQL table '{table_name}'!")


Database 'SQL_Project' already exists.
CSV file 'gold.dim_products.csv' has been successfully imported into PostgreSQL table 'gold_products'!


__Gold Customers Sales__ 

In [30]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# 1️⃣ File & database details
csv_file = "gold.fact_sales.csv"  # <-- apna actual path yahan daalo
db_name = "SQL_Project"
db_user = "postgres"   # apna PostgreSQL username
db_password = "Password"
db_host = "localhost"
db_port = "5432"
table_name = "gold_sales"  # lowercase table name, no quotes needed in SQL

# 2️⃣ Connect to default 'postgres' database to create new database if needed
conn = psycopg2.connect(
    user=db_user,
    password=db_password,
    host=db_host,
    port=db_port,
    dbname="postgres"
)
conn.autocommit = True
cur = conn.cursor()

# 3️⃣ Check if database exists
cur.execute(f"SELECT 1 FROM pg_database WHERE datname='{db_name}'")
exists = cur.fetchone()

# 4️⃣ Create database if it doesn't exist
if not exists:
    cur.execute(f'CREATE DATABASE "{db_name}"')
    print(f"Database '{db_name}' created!")
else:
    print(f"Database '{db_name}' already exists.")

cur.close()
conn.close()

# 5️⃣ Read CSV into pandas DataFrame
df = pd.read_csv(csv_file)

# 6️⃣ Ensure table name is lowercase
table_name = table_name.lower()

# 7️⃣ Create SQLAlchemy engine for the database
engine = create_engine(f"postgresql+psycopg2://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# 8️⃣ Export DataFrame to PostgreSQL
df.to_sql(table_name, engine, if_exists='replace', index=False)

print(f"CSV file '{csv_file}' has been successfully imported into PostgreSQL table '{table_name}'!")


Database 'SQL_Project' already exists.
CSV file 'gold.fact_sales.csv' has been successfully imported into PostgreSQL table 'gold_sales'!
